In [1]:
import dai
import pandas as pd

In [2]:
sd = '1990-01-01'
ed = '2026-12-31'

In [3]:
df = pd.DataFrame(pd.date_range(start='1990-01-01', end='2026-12-31', freq='D'), columns=['date'])
df['date'] = pd.to_datetime(df['date'])
df

,date
0,1990-01-01
1,1990-01-02
2,1990-01-03
3,1990-01-04
4,1990-01-05
...,...
13509,2026-12-27
13510,2026-12-28
13511,2026-12-29
13512,2026-12-30


In [4]:
df["dt_year"]      = df["date"].dt.year.astype(str)
df["dt_quarter"]   = df["date"].dt.to_period("Q").apply(lambda x: f"{x.year}{x.quarter:02}")
df["dt_month"]     = df["date"].dt.to_period("M").apply(lambda x: f"{x.year}{x.month:02}")
df["dt_fortnight"] = df["date"].apply(lambda x: f"{x.year}{((x.dayofyear - 1) // 14) + 1:02}")
df["dt_week"]      = df["date"].dt.strftime('%Y%U')
df["dt_day"]       = df["date"].dt.strftime("%Y%m%d")

df["is_year_start_trade"]      = (df["dt_year"] != df["dt_year"].shift(1)).astype(int)
df["is_year_end_trade"]        = (df["dt_year"] != df["dt_year"].shift(-1)).astype(int)

df["is_quarter_start_trade"]   = (df["dt_quarter"] != df["dt_quarter"].shift(1)).astype(int)
df["is_quarter_end_trade"]     = (df["dt_quarter"] != df["dt_quarter"].shift(-1)).astype(int)

df["is_month_start_trade"]     = (df["dt_month"] != df["dt_month"].shift(1)).astype(int)
df["is_month_end_trade"]       = (df["dt_month"] != df["dt_month"].shift(-1)).astype(int)

df["is_fortnight_start_trade"] = (df["dt_fortnight"] != df["dt_fortnight"].shift(1)).astype(int)
df["is_fortnight_end_trade"]   = (df["dt_fortnight"] != df["dt_fortnight"].shift(-1)).astype(int)

df["is_week_start_trade"]      = (df["dt_week"] != df["dt_week"].shift(1)).astype(int)
df["is_week_end_trade"]        = (df["dt_week"] != df["dt_week"].shift(-1)).astype(int)

df["is_year_start_order"]      = df["is_year_start_trade"].shift(-1, fill_value=0)
df["is_year_end_order"]        = df["is_year_end_trade"].shift(-1, fill_value=0)

df["is_quarter_start_order"]   = df["is_quarter_start_trade"].shift(-1, fill_value=0)
df["is_quarter_end_order"]     = df["is_quarter_end_trade"].shift(-1, fill_value=0)

df["is_month_start_order"]     = df["is_month_start_trade"].shift(-1, fill_value=0)
df["is_month_end_order"]       = df["is_month_end_trade"].shift(-1, fill_value=0)

df["is_fortnight_start_order"] = df["is_fortnight_start_trade"].shift(-1, fill_value=0)
df["is_fortnight_end_order"]   = df["is_fortnight_end_trade"].shift(-1, fill_value=0)

df["is_week_start_order"]      = df["is_week_start_trade"].shift(-1, fill_value=0)
df["is_week_end_order"]        = df["is_week_end_trade"].shift(-1, fill_value=0)

df["day_of_year_trade"]      = df.groupby("dt_year").cumcount() + 1
df["day_of_quarter_trade"]   = df.groupby("dt_quarter").cumcount() + 1
df["day_of_month_trade"]     = df.groupby("dt_month").cumcount() + 1
df["day_of_fortnight_trade"] = df.groupby("dt_fortnight").cumcount() + 1
df["day_of_week_trade"]      = df.groupby("dt_week").cumcount() + 1

df["day_of_year_trading_order"]      = df["day_of_year_trade"].shift(-1, fill_value=0)
df["day_of_quarter_trading_order"]   = df["day_of_quarter_trade"].shift(-1, fill_value=0)
df["day_of_month_trading_order"]     = df["day_of_month_trade"].shift(-1, fill_value=0)
df["day_of_fortnight_trading_order"] = df["day_of_fortnight_trade"].shift(-1, fill_value=0)
df["day_of_week_trading_order"]      = df["day_of_week_trade"].shift(-1, fill_value=0)

In [5]:
gap_from_prev = (df["date"] - df["date"].shift(1)).dt.days
gap_to_next   = (df["date"].shift(-1) - df["date"]).dt.days

df["is_holiday_begin_trade"] = (gap_to_next > 1).fillna(False).astype(int)
df["is_holiday_end_trade"]   = (gap_from_prev > 1).fillna(False).astype(int)

df["is_holiday_begin_order"] = df["is_holiday_begin_trade"].shift(-1, fill_value=0)
df["is_holiday_end_order"]   = df["is_holiday_end_trade"].shift(-1, fill_value=0)

In [6]:
df = df[1:]
df["is_holiday_end_order"].iloc[-1] = 1
df["is_holiday_begin_order"].iloc[-2] = 1
df

,date,dt_year,dt_quarter,dt_month,dt_fortnight,dt_week,dt_day,is_year_start_trade,is_year_end_trade,is_quarter_start_trade,...,day_of_week_trade,day_of_year_trading_order,day_of_quarter_trading_order,day_of_month_trading_order,day_of_fortnight_trading_order,day_of_week_trading_order,is_holiday_begin_trade,is_holiday_end_trade,is_holiday_begin_order,is_holiday_end_order
1,1990-01-02,1990,199001,199001,199001,199000,19900102,0,0,0,...,2,3,3,3,3,3,0,0,0,0
2,1990-01-03,1990,199001,199001,199001,199000,19900103,0,0,0,...,3,4,4,4,4,4,0,0,0,0
3,1990-01-04,1990,199001,199001,199001,199000,19900104,0,0,0,...,4,5,5,5,5,5,0,0,0,0
4,1990-01-05,1990,199001,199001,199001,199000,19900105,0,0,0,...,5,6,6,6,6,6,0,0,0,0
5,1990-01-06,1990,199001,199001,199001,199000,19900106,0,0,0,...,6,7,7,7,7,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13509,2026-12-27,2026,202604,202612,202626,202652,20261227,0,0,0,...,1,362,89,28,12,2,0,0,0,0
13510,2026-12-28,2026,202604,202612,202626,202652,20261228,0,0,0,...,2,363,90,29,13,3,0,0,0,0
13511,2026-12-29,2026,202604,202612,202626,202652,20261229,0,0,0,...,3,364,91,30,14,4,0,0,0,0
13512,2026-12-30,2026,202604,202612,202626,202652,20261230,0,0,0,...,4,365,92,31,1,5,0,0,1,0


In [7]:
# Save to DAI
ds = dai.DataSource("mldt_cn_natural_trading_calendar_daily")
def update_df(df_new):
    df_new = df
    return df_new
ds.apply_bdb(update_df, as_type=pd.DataFrame)

dai.DataSource.write_bdb(
    data=df,
    id="mldt_cn_natural_trading_calendar_daily",
    unique_together=["date"],
    indexes=["date"],
)

dai.DataSource("mldt_cn_natural_trading_calendar_daily")